## This notebook reads OSM data and creates a grid of cells tagged by their primary land use or road type label.

In [15]:
import numpy as np
import csv
import pandas as pd

from shapely.geometry import Point, MultiPolygon, Polygon, mapping
import shapely
import folium
import json

from constants import Constants
from osm_reader import OSMReader
from grid_definition import GridDefinition

grid_definition = GridDefinition()
grid_definition.init()
constants = Constants()
GRID_SIZE = constants.getGridSize()

osm_reader = OSMReader()
osm_reader.init()
constants = Constants()

## Retrieve road and land use classes and map them as lines and polygons, respectively

In [16]:
roadGeos = osm_reader.getRoadGeoClasses()
roadType = folium.Map(location=[55.943, -3.19], zoom_start=15,tiles="Stamen Toner")
# print(len(roadGeos))
for num, roadGeo in enumerate(roadGeos):
    print(num, roadGeo['features'])
    osm_reader.addRoadType(roadType, roadGeo, num)
roadType.save("../images/maps/roadTypeStamenToner_wt3.html")
    
areaGeos = osm_reader.getLandGeoClasses()
landUse = folium.Map(location=[55.943, -3.19], zoom_start=15, tiles="Stamen Toner")
for num, areaGeo in enumerate(areaGeos):
    osm_reader.addLandUse(landUse, areaGeo, num)        
landUse.save("../images/maps/landUseTiles.html")

0 [{'type': 'Feature', 'properties': {'osm_id': '2956332', 'name': 'Nicolson Street', 'highway': 'primary', 'z_order': 7, 'other_tags': '"lit"=>"yes","maxspeed"=>"20 mph","ref"=>"A7","surface"=>"asphalt"'}, 'geometry': {'type': 'LineString', 'coordinates': [[-3.1835257, 55.9438013], [-3.1835704, 55.9439324], [-3.183719, 55.9443146], [-3.183749, 55.9443964], [-3.183762, 55.9444287], [-3.1838277, 55.9446149], [-3.1838542, 55.9447084], [-3.1838953, 55.9448028], [-3.1839742, 55.9448891], [-3.1840428, 55.9449648], [-3.1842037, 55.9451214]]}}, {'type': 'Feature', 'properties': {'osm_id': '2970983', 'name': 'Melville Drive', 'highway': 'primary', 'z_order': 7, 'other_tags': '"lanes"=>"2","lit"=>"yes","maxspeed"=>"20 mph","ref"=>"A700","surface"=>"asphalt"'}, 'geometry': {'type': 'LineString', 'coordinates': [[-3.1827389, 55.9404775], [-3.1828929, 55.9404559], [-3.1830342, 55.9404304], [-3.1833632, 55.9403495], [-3.1836608, 55.9402703], [-3.1839537, 55.9401869], [-3.1843756, 55.9400697], [-3.1

IndexError: list index out of range

In [ ]:
roadType

In [ ]:
landUse

In [17]:
corners = corners = constants.getCorners()

topLat = corners[0][0]
bottomLat = corners[2][0]

leftLon = corners[0][1]
rightLon = corners[1][1]

height = topLat - bottomLat
width = rightLon - leftLon
heightInterval = height / GRID_SIZE
widthInterval = width / GRID_SIZE

In [18]:
cells = osm_reader.getGeoCells()

## Map road type from csv

In [19]:
osm_dir = "/Users/ryanegan/Documents/diss/projectZoe/data/osm/"
filename = osm_dir + "roadtype_grid" + str(GRID_SIZE) + ".csv"
      
roadPixels = folium.Map(location=[55.943, -3.19],tiles="Stamen Toner", zoom_start=15)
pd_df=pd.read_csv(filename, sep=',',header=None, skiprows=1)
road_types = pd_df.values

for cell_num, cell in enumerate(cells):
    cell_multi = cell[0]["geometry"]
    cell_shape = shapely.geometry.asShape(cell_multi)
    row = int(cell_num / GRID_SIZE)
    col = int(cell_num % GRID_SIZE)
    osm_reader.addLandUse(roadPixels, cell_multi, int(road_types[row][col]))

#roadPixels.save("../images/maps/roadType.html")
roadPixels  

FileNotFoundError: [Errno 2] File b'/Users/ryanegan/Documents/diss/projectZoe/data/osm/roadtype_grid50.csv' does not exist: b'/Users/ryanegan/Documents/diss/projectZoe/data/osm/roadtype_grid50.csv'

## Map Land Use from CSV

In [ ]:
osm_dir = "/Users/zoepetard/Google Drive/Edinburgh/MscProj/FillingTheGaps/data/osm/"
filename = osm_dir + "landuse_grid" + str(GRID_SIZE) + ".csv"
        
landPixels = folium.Map(location=[55.943, -3.19],tiles="Stamen Toner", zoom_start=15)
pd_df=pd.read_csv(filename, sep=',',header=None, skiprows=1)
land_use = pd_df.values

for cell_num, cell in enumerate(cells):
    cell_multi = cell[0]["geometry"]
    cell_shape = shapely.geometry.asShape(cell_multi)
    row = int(cell_num / GRID_SIZE)
    col = int(cell_num % GRID_SIZE)
    index = int(land_use[row][col])
    osm_reader.addLandUse(landPixels, cell_multi, index)

#landPixels.save("../images/maps/landUse.html")
landPixels

## Make file with just green land use

In [10]:
osm_dir = "/Users/zoepetard/Google Drive/Edinburgh/MscProj/FillingTheGaps/data/osm/"
filename = osm_dir + "landuse_grid" + str(GRID_SIZE) + ".csv"

pd_df=pd.read_csv(filename, sep=',',header=None, skiprows=1)

land_use = pd_df.values

cells = osm_reader.getGeoCells()


for cell_num, cell in enumerate(cells):
    cell_multi = cell[0]["geometry"]
    cell_shape = shapely.geometry.asShape(cell_multi)
    row = int(cell_num / GRID_SIZE)
    col = int(cell_num % GRID_SIZE)
    index = int(land_use[row][col])
    if (index ==3):
        land_use[row][col] = 1
    else:
        land_use[row][col] = 0
        
osm_dir = "/Users/zoepetard/Google Drive/Edinburgh/MscProj/FillingTheGaps/data/osm/"
filename = osm_dir + "landuse_green_grid" + str(GRID_SIZE) + ".csv"
with open(filename, 'w') as csvfile:
    csvfile.write("0=not green, 1=green \n")
    writer = csv.writer(csvfile, lineterminator='\n')
    writer.writerows(land_use)        


FileNotFoundError: [Errno 2] File b'/Users/zoepetard/Google Drive/Edinburgh/MscProj/FillingTheGaps/data/osm/landuse_grid50.csv' does not exist: b'/Users/zoepetard/Google Drive/Edinburgh/MscProj/FillingTheGaps/data/osm/landuse_grid50.csv'

## Label cell geometry objects with major land use

In [4]:
## Only needs to be done once per test area
for cell_num, cell in enumerate(cells):
    cell_multi = cell[0]["geometry"]
    cell_shape = shapely.geometry.asShape(cell_multi)
    
    LU_areas = []
    major_LU = 0
    major_LU_index = 0

    for LU_level, areaGeo in enumerate(areaGeos):
        features = areaGeo["features"]
        #print(len(features))
        area = 0
        for feat in range(len(features)):
            landuse_multi = areaGeo["features"][feat]["geometry"] 
            landuse_shape = shapely.geometry.asShape(landuse_multi)
            intersection = cell_shape.intersection(landuse_shape)
            area += intersection.area
        LU_areas.append(area)
        if (area > major_LU):
            major_LU = area
            major_LU_index = LU_level
    if (major_LU_index == 0): #if there are no landuses in the cell, make it the middle land use category
        major_LU_index = 2
    #osm_reader.addLandUse(pixelMap, cell_multi, major_LU_index)
    cells[cell_num][0]["properties"]["major_LU"] = major_LU_index
    
#iterate over first 0/1 to go through diff landuse collections (levels 1 to 6)
#iterate over second 0/1 to go through diff landuse items (items within a level)


NameError: name 'cells' is not defined

## Write cell major land use grid to csv for future retrieval

In [3]:
## Only needs to be done once per test area
cells = osm_reader.getCellsWithMajorLU()
cells_majorLU = np.zeros((GRID_SIZE, GRID_SIZE))

for cell_num, cell in enumerate(cells):
    #cell_multi = cell[0]["geometry"]
    major_LU = cell[0]["properties"]["major_LU"] #= major_LU_index
    row = int(cell_num / GRID_SIZE)
    col = int(cell_num % GRID_SIZE)
    cells_majorLU[row][col] = major_LU

osm_dir = "/Users/ryanegan/Documents/diss/projectZoe/data/osm/"
filename = osm_dir + "landuse_grid" + str(GRID_SIZE) + ".csv"
with open(filename, 'w') as csvfile:
    csvfile.write("1=commercial, 2=residential, 3=green" + "\n")
    writer = csv.writer(csvfile, lineterminator='\n')
    writer.writerows(cells_majorLU)

## Label cell geometry objects with major road type.

In [21]:
## Only needs to be done once per test area
for cell_num, cell in enumerate(cells):
    cell_multi = cell[0]["geometry"]
    cell_shape = shapely.geometry.asShape(cell_multi)
    
    for road_level, roadGeo in enumerate(roadGeos):
        cells[cell_num][0]["properties"]["major_road"] = 5 #As a default, road level is lowest
        intersection = False 
        features = roadGeo["features"]
        for feat in range(len(features)):
            roadtype_multi = roadGeo["features"][feat]["geometry"] 
            roadtype_shape = shapely.geometry.asShape(roadtype_multi)
            intersection = cell_shape.intersects(roadtype_shape)
            if (intersection):
                cells[cell_num][0]["properties"]["major_road"] = road_level
                break
        if (intersection):
            break

## Write cell major road type grid to csv for future retrieval

In [22]:
## Only needs to be done once per test area
cells_major_road = np.zeros((GRID_SIZE, GRID_SIZE))

for cell_num, cell in enumerate(cells):
    major_road = cell[0]["properties"]["major_road"] 
    row = int(cell_num / GRID_SIZE)
    col = int(cell_num % GRID_SIZE)
    cells_major_road[row][col] = major_road

osm_dir = "/Users/ryanegan/Documents/diss/projectZoe/data/osm/"
filename = osm_dir + "roadtype_grid" + str(GRID_SIZE) + ".csv"
with open(filename, 'w') as csvfile:
    csvfile.write("0=primary, 1=secondary, 2=tertiary, 3=unclassified, 4=residential/service, 5=pedestrian/cycle/noroad" + "\n")
    writer = csv.writer(csvfile, lineterminator='\n')
    writer.writerows(cells_major_road)